In [6]:
import os
import pandas as pd
from dotenv import load_dotenv

if os.getcwd().split("/")[-1] == "sandbox":
    os.chdir("../")
    
load_dotenv()

True

In [7]:
from food_co2_estimator.chains.recipe_extractor import get_recipe_extractor_chain
from food_co2_estimator.utils.url import get_url_text
recipe_extractor = get_recipe_extractor_chain(verbose=False)

In [8]:
from food_co2_estimator.utils.url import get_url_text

from food_co2_estimator.output_parsers.recipe_extractor import Recipe


url = "https://www.valdemarsro.dk/frikadeller/"
text = get_url_text(url)
recipe_extractor = get_recipe_extractor_chain(verbose=False)
recipe: Recipe = recipe_extractor.invoke(input=text)

In [9]:
from food_co2_estimator.chains.rag_co2_estimator import rag_co2_emission_chain
from food_co2_estimator.chains.translator import get_translation_chain

In [10]:
emission_chain = rag_co2_emission_chain(verbose=True)
translation_chain = get_translation_chain()

emission_chain_with_translation = translation_chain | emission_chain

emissions = emission_chain_with_translation.invoke(
    input={"inputs": recipe.ingredients, "language": "da"}
)

In [11]:
emissions

CO2Emissions(emissions=[CO2perKg(ingredient='500 g minced pork', comment="Best match is 'Pork, minced, 15-20% fat' with emissions of 3.7 kg CO2e / kg.", unit='kg CO2e / kg', co2_per_kg=3.7), CO2perKg(ingredient='2 tablespoons wheat flour', comment="Best match is 'Wheat flour, graham flour, whole grain' with emissions of 1.2 kg CO2e / kg.", unit='kg CO2e / kg', co2_per_kg=1.2), CO2perKg(ingredient='2 tablespoons finely rolled oatmeal', comment="Best match is 'Oatmeal, not enriched' with emissions of 1.9 kg CO2e / kg.", unit='kg CO2e / kg', co2_per_kg=1.9), CO2perKg(ingredient='1 cup milk', comment="Best match is 'Sweet milk, 3.5% fat' with emissions of 0.5 kg CO2e / kg.", unit='kg CO2e / kg', co2_per_kg=0.5), CO2perKg(ingredient='1 egg white', comment="Best match is 'Eggs, scrambled eggs' with emissions of 1.0 kg CO2e / kg.", unit='kg CO2e / kg', co2_per_kg=1.0), CO2perKg(ingredient='1 egg', comment="Best match is 'Eggs, scrambled eggs' with emissions of 1.0 kg CO2e / kg.", unit='kg CO2

In [12]:
emission_chain.invoke(["1 egg"])

CO2Emissions(emissions=[CO2perKg(ingredient='1 egg', comment="Best match is 'Eggs, scrambled eggs' as it is the closest processing level.", unit='kg CO2e / kg', co2_per_kg=1.0)])

In [13]:
from food_co2_estimator.retrievers.emission_retriever import get_emission_retriever_chain


retriever = get_emission_retriever_chain(k=10)

retriever.invoke("1 egg")

{'Eggs, scrambled eggs': '1.0 kg CO2e / kg',
 'Egg salad': '1.5 kg CO2e / kg',
 'Noodles, eggs': '1.7 kg CO2e / kg'}

async test

In [ ]:

recipe_extractor_chain = get_url_text | recipe_extractor
full_chain = recipe_extractor | emission_chain_with_translation


recipe = await recipe_extractor_chain.ainvoke(url)
emissions = await emission_chain_with_translation.ainvoke(
    input={"inputs": recipe.ingredients, "language": "da"}
)

In [ ]:
emissions